In [1]:
import os

import numpy as np
from sklearn.metrics import matthews_corrcoef
from sklearn.neighbors import KNeighborsClassifier
from torch.utils.data import DataLoader, Subset

from datasets.embeddings_localization_dataset import EmbeddingsLocalizationDataset
from torchvision.transforms import transforms

from datasets.transforms import SolubilityToInt
from utils.general import numpy_collate_for_reduced

os.chdir('C:/Users/HannesStark\\projects\\protein-localization')
#train_embeddings = 'data/embeddings/train_T5_reduced.h5'
#train_remapping = 'data/fasta_files/train.fasta'
#
#val_embeddings = 'data/embeddings/test_T5_reduced.h5'
#val_remapping = 'data/fasta_files/test_as_per_deeploc.fasta'

#train_embeddings = 'data/embeddings/train_T5_reduced.h5'
#train_remapping = 'data/fasta_files/train.fasta'
#val_embeddings = 'data/embeddings/new_hard_set_t5_reduced.h5'
#val_remapping = 'data/embeddings/new_hard_set_t5_remapping.fasta'

#train_embeddings = 'data/embeddings/train_reduced.h5'
#train_remapping = 'data/embeddings/train_remapped.fasta'
#val_embeddings = 'data/embeddings/new_hard_set_BERT_reduced.h5'
#val_remapping = 'data/embeddings/new_hard_set_BERT_remapping.fasta'

#train_embeddings = 'data/embeddings/train_reduced.h5'
#train_remapping = 'data/embeddings/train_remapped.fasta'
#
#val_embeddings = 'data/embeddings/test_reduced.h5'
#val_remapping = 'data/embeddings/test_remapped.fasta'
#
#
#train_embeddings = 'data/seqvec_reduced/train_reduced.h5'
#train_remapping = 'data/seqvec_reduced/train_remapped.fasta'
#
#val_embeddings = 'data/seqvec_reduced/test_reduced.h5'
#val_remapping = 'data/seqvec_reduced/test_remapped.fasta'

# For Uniref 50:
#train_embeddings= 'data/uniref_embeddings_pooled/train.h5'
#train_remapping= 'data/uniref_embeddings_pooled/train.fasta'
#
#val_embeddings= 'data/uniref_embeddings_pooled/new_hard_set.h5'
#val_remapping= 'data/uniref_embeddings_pooled/new_hard_set.fasta'
#key_format= 'fasta_descriptor'

#unirep
train_embeddings = 'data/bepler/train/reduced_embeddings_file.h5'
train_remapping = 'data/bepler/train/remapped_sequences_file.fasta'
val_embeddings= 'data/bepler/deeploc_test////reduced_embeddings_file.h5'
val_remapping= 'data/bepler/deeploc_test/////remapped_sequences_file.fasta'
key_format= 'hash'

#val_embeddings = 'data/uniref_embeddings_pooled/test_as_per_deeploc.h5'
#val_remapping = 'data/uniref_embeddings_pooled/test_as_per_deeploc.fasta'
#key_format= 'fasta_descriptor'

descriptions_with_hash = False

In [2]:

transform = transforms.Compose([SolubilityToInt()])
train_set = EmbeddingsLocalizationDataset(train_embeddings, train_remapping,
                                             unknown_solubility=True,
                                             key_format=key_format,
                                             embedding_mode='lm',
                                             transform=transform)
val_set = EmbeddingsLocalizationDataset(val_embeddings, val_remapping,
                                             unknown_solubility=True,
                                             key_format=key_format,
                                             embedding_mode='lm',
                                             transform=transform)

train_loader = DataLoader(train_set, batch_size=len(train_set), collate_fn=numpy_collate_for_reduced)
val_loader = DataLoader(val_set, batch_size=len(val_set), collate_fn=numpy_collate_for_reduced)

train_data = next(iter(train_loader))  # tuple of embedding, localization, solubility, metadata
val_data = next(iter(val_loader))  # tuple of embedding, localization, solubility, metadata
classifier = KNeighborsClassifier(n_neighbors=1, p=1)
#classifier = KNeighborsClassifier(n_neighbors=1, metric='cosine')
classifier.fit(train_data[0], train_data[1])


KNeighborsClassifier(n_neighbors=1, p=1)

In [3]:
predictions = classifier.predict(val_data[0])
distances, _ = classifier.kneighbors(val_data[0])
#distances = np.array(distances).sum(-1).squeeze()




In [4]:
import matplotlib

import seaborn as sns
sns.set_theme()
import pandas as pd
import matplotlib.pyplot as plt
#matplotlib.rcParams['figure.dpi'] = 300
matplotlib.rcParams['savefig.dpi'] = 300

In [5]:
# Get Accuracy and MCC
print(len(predictions))
print(100 * np.equal(predictions, val_data[1]).sum() / len(predictions))
matthews_corrcoef(val_data[1], predictions)

431
25.986078886310906
[6 6 6 3 1 1 2 9 1 6 1 6 0 3 0 5 6 6 1 1 6 1 6 6 1 1 1 0 6 2 5 2 6 6 1 3 6
 3 6 3 5 3 1 1 6 1 6 6 6 6 3 1 5 5 1 1 6 6 1 5 1 1 9 6 0 1 0 6 6 6 6 6 1 1
 0 1 4 1 3 1 4 1 6 6 8 6 4 1 6 1 5 6 1 6 6 6 6 1 5 1 0 4 6 6 6 2 6 5 1 5 6
 6 1 6 5 5 3 1 6 1 5 0 5 6 9 6 6 6 6 2 4 1 5 6 6 6 6 6 6 5 2 5 1 6 6 4 8 6
 6 0 0 6 0 1 6 6 6 1 0 9 5 6 0 1 8 6 5 6 1 6 6 5 1 6 0 5 6 1 7 6 5 1 5 6 4
 0 6 1 6 1 9 5 0 1 1 1 1 0 0 6 6 8 0 1 1 8 1 6 6 4 3 6 6 1 5 1 9 8 6 1 1 1
 2 5 5 4 5 0 0 1 6 6 6 6 6 5 8 0 1 1 6 2 6 0 6 4 6 4 1 1 1 6 2 6 0 9 6 6 2
 6 8 2 1 9 5 8 6 6 6 9 1 1 3 6 0 5 6 0 5 9 6 3 6 9 6 4 2 6 3 5 0 1 8 5 5 7
 1 0 6 5 0 0 1 9 6 5 6 6 3 6 6 6 9 5 6 5 6 6 5 5 2 8 9 1 2 6 0 9 6 6 2 9 6
 6 6 5 6 0 5 9 6 6 9 3 1 8 6 4 9 6 9 2 9 6 9 5 5 1 3 5 7 2 1 6 9 6 5 6 3 4
 8 2 6 9 9 2 2 6 5 5 3 5 5 6 5 0 6 9 6 9 6 9 2 6 5 6 9 3 5 9 9 5 5 9 9 6 5
 5 9 5 5 6 5 9 2 6 9 9 6 9 1 6 5 0 6 9 0 6 9 9 6]
[array(8), array(3), array(1), array(0), array(3), array(1), array(0), array(9), array(0), array(1), a

In [13]:
# majority classifier:
majority_prediction = np.ones_like(predictions, dtype=int)*np.argmax(np.bincount(val_data[1]))
matthews_corrcoef(val_data[1], majority_prediction)

[array(8), array(3), array(1), array(0), array(3), array(1), array(0), array(9), array(0), array(1), array(1), array(0), array(4), array(1), array(2), array(0), array(1), array(0), array(4), array(4), array(6), array(9), array(1), array(6), array(1), array(1), array(1), array(3), array(6), array(1), array(1), array(7), array(0), array(1), array(0), array(1), array(6), array(1), array(6), array(0), array(1), array(6), array(1), array(1), array(6), array(0), array(9), array(6), array(9), array(1), array(0), array(0), array(9), array(6), array(1), array(0), array(6), array(1), array(1), array(3), array(6), array(9), array(0), array(1), array(1), array(6), array(6), array(4), array(1), array(1), array(1), array(0), array(1), array(0), array(9), array(6), array(1), array(0), array(2), array(6), array(0), array(6), array(6), array(6), array(1), array(6), array(3), array(0), array(6), array(6), array(9), array(4), array(3), array(1), array(0), array(1), array(6), array(1), array(0), array(7)]

C:\Users\HannesStark\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:870: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


0.0

In [6]:
cutoffs = np.linspace(distances.min(),distances.max(),1000)
results = np.array([predictions,val_data[1], distances.squeeze()]).T


best_accuracy = 0
denovo_accuracies = []
knn_accuracies = []
accuracies = []
mccs = []
percentage_denovo = []
for cutoff in cutoffs:
    indices = results[:, 2] <= cutoff
    below_cutoff = results[indices]
    chosen_denovo_predictions = denovo_predictions[np.invert(indices)]
    knn_accuracies.append(100 * np.equal(below_cutoff[:, 0], below_cutoff[:, 1]).sum() / len(below_cutoff))
    denovo_accuracies.append(100 * np.equal(chosen_denovo_predictions[:, 0], chosen_denovo_predictions[:, 1]).sum() / len(chosen_denovo_predictions))
    combined_results = np.concatenate([chosen_denovo_predictions[:, :2], below_cutoff[:, :2]])
    accuracy = 100*np.equal(combined_results[:, 0], combined_results[:, 1]).sum() / len(combined_results)
    accuracies.append(accuracy)
    mccs.append(matthews_corrcoef(combined_results[:, 1], combined_results[:, 0]))
    percentage_denovo.append(100*len(chosen_denovo_predictions)/len(results))
    if accuracy >= best_accuracy:
        best_accuracy = accuracy

print('only Light attention accuracy: ',100 * np.equal(denovo_predictions[:, 0], denovo_predictions[:, 1]).sum() / len(denovo_predictions))
print('best accuracy: ', best_accuracy)
best_accuracy_index = np.array(accuracies).argmax()
print('knn accuracy: ', knn_accuracies[best_accuracy_index])
print('denovo accuracy: ', denovo_accuracies[best_accuracy_index])
print('percentage of denovo predictions: ', percentage_denovo[best_accuracy_index])
print('best_distance_cutoff: ', cutoffs[best_accuracy_index])
print('knn only accuracy: ', knn_accuracies[-1])


IndexError: boolean index did not match indexed array along dimension 0; dimension is 2768 but corresponding boolean dimension is 431

In [ ]:
plt.rcParams['figure.figsize'] = [10, 8]

df = pd.DataFrame(np.array([cutoffs, knn_accuracies]).T,
                  columns=["distance",'knn_accuracies'])
df = df.melt('distance', var_name='cols',  value_name='percent')
sns.lineplot(x="distance", y="percent", hue='cols', data=df)
plt.legend(labels=['1-NN accuracy'])
plt.xlim(0,25)
plt.ylim(0)
plt.ylabel('Accuracy')
plt.xlabel('Euclidean distance')
plt.savefig('figure1.png')
plt.show()
plt.clf()

df = pd.DataFrame(np.array([cutoffs,percentage_denovo, denovo_accuracies,]).T,
                  columns=["distance",'percentage_denovo','denovo_accuracies',])
df = df.melt('distance', var_name='cols',  value_name='percent')
sns.lineplot(x="distance", y="percent", hue='cols', data=df)
plt.legend(labels=['percentage LA','LA accuracy'])
plt.xlim(0,25)
plt.ylim(0)
plt.ylabel('Accuracy')
plt.xlabel('Euclidean distance')
plt.savefig('figure2.png')
plt.show()
plt.clf()

df = pd.DataFrame(np.array([cutoffs, knn_accuracies, denovo_accuracies, accuracies, percentage_denovo]).T,
                  columns=["distance", 'knn_accuracies', 'denovo_accuracies',"accuracies", 'percentage_denovo'])
df = df.melt('distance', var_name='cols',  value_name='percent')
sns.lineplot(x="distance", y="percent", hue='cols', data=df)
plt.legend(labels=['1NN accuracy', 'LA accuracy','combined accuracy', 'percentage LA'])
plt.xlim(0,25)
plt.ylim(0)
plt.xlabel('Euclidean distance')
plt.savefig('figure3.png')
plt.show()
plt.clf()

df = pd.DataFrame(np.array([cutoffs,accuracies, knn_accuracies, denovo_accuracies]).T,
                  columns=["distance", "accuracies", 'knn_accuracies', 'denovo_accuracies'])
df = df.melt('distance', var_name='cols',  value_name='percent')
sns.lineplot(x="distance", y="percent", hue='cols', data=df)
plt.legend(labels=['combined accuracy', '1NN accuracy', 'LA accuracy', 'percentage LA'])
plt.savefig('figure4.png')


